# Caderno para construção e teste do BOT

## Loading

In [1]:
## Loader do PDF
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

path = "data\\manual_consolidado.pdf"

## Loaders structured and unstructured

#loader_un = UnstructuredPDFLoader(path)
loader_structured = PyMuPDFLoader(path)

pages_str = loader_structured.load()
#pages_str = loader_un.load()

## Cleaning

In [2]:
from langchain.text_splitter import CharacterTextSplitter

## Criando texto splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
    )

## Split
texts = text_splitter.split_documents(pages_str)




In [3]:
### Testanto

texts[500].page_content

'O sistema gera uma confirmação da solicitação, clicar em “Ok”.\nO status final do processo fica como “Lote entregue”. Após a finalização, o processo não pode\nmais ser excluído e o botão “Excluir” fica indisponível.\nO novo lote pode ser consultado na aba “Lote fornecedor”, com uma nova sequência que\nfoi criada. O lote que deu origem ao processo de unitarização segue com o status\n“Enviado para unitarização”.\nConsultores:\nBruno Emanoel Calixto Ferreira\nUllysses Henrique Machado Ribas\nPágina:\n75 de 80'

In [4]:
## Função para remover espaço em branco

def remove_ws(d):
    text = d.page_content.replace("\n", "")
    d.page_content = text
    return d

# Clean texts by removing whitespace from each document
texts_cleaned = [remove_ws(d) for d in texts]

In [5]:
texts_cleaned[500].page_content

len(texts), len(texts_cleaned)

(5458, 5458)

In [9]:
## openai

# config.py
import os
from dotenv import load_dotenv

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path="env/.env")

# Acessar a chave da OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Retriever

In [14]:
## retriever openai

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# uses OpenAI embeddings to build a retriever
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
# Creates the document retriever using docs and embeddings
db = FAISS.from_documents(texts_cleaned, embeddings)



# Asking the retriever to do similarity search based on Query
#query = "Foreign Aid for Lowari Road Tunnel & Access Roads Project (2nd Revised )"
#answer = db.similarity_search(query)

# Building the retriever
retriever = db.as_retriever(search_kwargs={'k': 3})


In [17]:
import faiss
import pickle

# Salvar o índice FAISS
faiss.write_index(db.index, 'data/faiss_index.index')

# Salvar os documentos associados (metadados)
with open('data/faiss_docs.pkl', 'wb') as f:
    pickle.dump(db.docstore._dict, f)



In [ ]:
import faiss
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Carregar o índice FAISS
index = faiss.read_index('data/faiss_index.index')

# Carregar os documentos (metadados)
with open('data/faiss_docs.pkl', 'rb') as f:
    docstore = pickle.load(f)

# Recriar o OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Recriar o FAISS VectorStore usando o índice e os documentos
db = FAISS(embedding_function=embeddings, index=index, docstore=docstore)

# Recriar o retriever
retriever = db.as_retriever(search_kwargs={'k': 3})



## Chain and chats

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Função que o bot usará para responder a mensagens diretas

template = """
Você é um assistente especializado no sistema de gestão hospitalar Tasy, utilizado pela Fundação Hospitalar do Estado de Minas Gerais (FHEMIG).
Seu objetivo é ajudar os usuários a utilizarem o Tasy de forma eficiente, respondendo a perguntas com base nos manuais e documentos internos disponíveis.

### Instruções:

1. Sempre forneça uma resposta clara e objetiva com base nas informações extraídas dos manuais.
2. Evite raciocínios longos ou desnecessários, forneça respostas diretas e organizadas em passos claros.
3. Se necessário, você pode buscar em múltiplos documentos para encontrar a melhor resposta.
4. Caso a resposta envolva várias etapas, organize-as de forma numerada.
5. Se o usuário pedir mais detalhes, você pode expandir a explicação ou oferecer links para seções específicas dos manuais.

### Estrutura de Resposta:
1. **Pergunta**: Exiba a pergunta que o usuário fez.
2. **Resumo Rápido**: Forneça uma resposta resumida em uma ou duas frases.
3. **Passos Detalhados**: Se o problema envolver etapas, forneça uma lista numerada de ações.
4. **Referência ao Manual**: Sempre que possível, inclua referências ao manual, como o número da página ou seção relevante.

### Exemplo de Pergunta e Resposta:

**Pergunta**: Como cadastrar um motivo de interdição de leito no Tasy?

**Resposta**:

1. **Resumo Rápido**: Para cadastrar um motivo de interdição de leito, acesse "Cadastros Gerais" e selecione "Motivo Interdição leito".

2. **Passos Detalhados**:
   - Acesse o menu principal do Tasy.
   - Clique em “Cadastros Gerais”.
   - Digite ou selecione “Motivo Interdição leito”.
   - Preencha os campos obrigatórios com as informações relevantes (como descrição do motivo).
   - Salve as informações para concluir o cadastro.

3. **Referência ao Manual**: Consulte o manual do Tasy, seção relacionada ao cadastro de motivos, na página 1067 para mais informações detalhadas.

{context}
"""

# Converter o template em prompt_template

prompt = ChatPromptTemplate.from_template(template)


## Configurar llm com Ollama

llm = ChatOpenAI(model="gpt-4o-mini")

## Configurar chain

chain = chain = (
# The initial dictionary uses the retriever and user supplied query
    {"context":retriever,
     "query":RunnablePassthrough()}
# Feeds that context and query into the prompt then model & lastly 
# uses the ouput parser, do query for the data.
    |  prompt  | llm | StrOutputParser()
)



## Testando!

In [28]:
response = chain.invoke("Fale-me sobre o módulo para urgência")
formatted_response = response.replace("\n", "\n\n")  # Adiciona uma linha em branco após cada nova linha

print(formatted_response)


**Pergunta**: Como cadastrar um novo paciente no Tasy?



**Resposta**:



1. **Resumo Rápido**: Para cadastrar um novo paciente no Tasy, você deve acessar a função de cadastro de pacientes e preencher os dados necessários.



2. **Passos Detalhados**:

   - Acesse o menu principal do Tasy.

   - Clique em “Cadastros” e depois em “Paciente”.

   - Selecione a opção “Novo” para iniciar o cadastro.

   - Preencha os campos obrigatórios, como nome, data de nascimento, e documentos de identificação.

   - Revise as informações inseridas e clique em “Salvar” para concluir o cadastro.



3. **Referência ao Manual**: Para mais detalhes, consulte o manual do Tasy na seção de cadastro de pacientes, página 245 para informações adicionais.
